In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
%matplotlib inline

/home/teguh/.local/lib/python2.7/site-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/home/teguh/.local/lib/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/home/teguh/.local/lib/python2.7/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/home/teguh/.local/lib/python2.7/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/home/teguh/.local/lib/python2.7/site-packages/pandas/core/indexes/base.py:7: RuntimeWarning: n

In [2]:
from scipy.io import loadmat
data_train = loadmat('data/sarcos_inv.mat')
data_test = loadmat('data/sarcos_inv_test.mat')

x_train = data_train['sarcos_inv'][:,:21]
y_train = data_train['sarcos_inv'][:,21:22]
y_var = np.var(y_train)

x_test = data_test['sarcos_inv_test'][:,:21]
y_test = data_test['sarcos_inv_test'][:,21:22]

In [3]:
x_train_cos = np.cos(x_train)
x_train_sin = np.sin(x_train)
x_train_fourier = np.hstack([x_train_cos, x_train_sin])

In [4]:
x_test_cos = np.cos(x_test)
x_test_sin = np.sin(x_test)
x_test_fourier = np.hstack([x_test_cos, x_test_sin])

### Fourier Linear Regression

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_model = LinearRegression()
lin_model.fit(x_train_fourier, y_train)

/home/teguh/.local/lib/python2.7/site-packages/sklearn/utils/__init__.py:9: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .murmurhash import murmurhash3_32
/home/teguh/.local/lib/python2.7/site-packages/sklearn/utils/extmath.py:24: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._logistic_sigmoid import _log_logistic_sigmoid
/home/teguh/.local/lib/python2.7/site-packages/sklearn/utils/extmath.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .sparsefuncs_fast import csr_row_norms
/home/teguh/.local/lib/python2.7/site-packages/sklearn/linear_model/base.py:35: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ..utils.seq_dataset import ArrayDataset, CSRDataset
/home/teguh/.local/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:23: RuntimeW

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [6]:
y_train_predict = lin_model.predict(x_train_fourier)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_predict))

y_test_predict = lin_model.predict(x_test_fourier)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_predict))

print rmse_train**2/y_var, rmse_test**2/y_var

0.6534460287369542 0.64445456770569


### Using polynomial expansion

In [7]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree = 3)

x_fourier_poly_train = poly.fit_transform(x_train_fourier)
x_fourier_poly_test = poly.fit_transform(x_test_fourier)

poly3 = PolynomialFeatures(degree = 3)

x_linear_poly_train = poly3.fit_transform(x_train)
x_linear_poly_test = poly3.fit_transform(x_test)

x_train_poly = np.hstack([x_linear_poly_train, x_fourier_poly_train])
x_test_poly = np.hstack([x_linear_poly_test, x_fourier_poly_test])

print x_train.shape, x_train_poly.shape

(44484, 21) (44484, 16214)


In [8]:
lin_model.fit(x_train_poly, y_train)

y_train_predicted_poly = lin_model.predict(x_train_poly)
y_test_predicted_poly = lin_model.predict(x_test_poly)

rmse_train_poly = np.sqrt(mean_squared_error(y_train, y_train_predicted_poly))
rmse_test_poly = np.sqrt(mean_squared_error(y_test, y_test_predicted_poly))

print rmse_train_poly**2/y_var, rmse_test_poly**2/y_var

0.005378130949044351 0.005248815741764596


In [ ]:
degree 2, 3: 0.012455334701432008 0.01189985340208558